*SU(4) to SU(2)_S x SU(2)_T decomposition*  

In [15]:
#########################################################################################################
## Calculate  SU(4) to SU(2)_S x SU(2)_T decomposition                                                 ##
##                                                                                                     ##
## Args:                                                                                               ## 
##       irrep (4 non-increasing non-negative integers:f1,f2,f3,f4)                                    ##
##                                                                                                     ##
##    Returns:                                                                                         ##
##        DataFrame with quantum numbers and dimensions                                                ##
##                                                                                                     ##
##    Developed by J.P. Valencia Valencia, S. Cordoba and R. Henao                                     ## 
#########################################################################################################  
import os, sys
module_path = os.path.abspath("/home/pvalen/su4-branching/su4_branching") #<---use the path to modules
if module_path not in sys.path:
    sys.path.append(module_path)
import su4_branching 
print("✓ Module imported successfully!")
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)
 
from su4_export import export_su4_with_labels, export_multiple_representations
from pathlib import Path
from typing import Tuple
from fractions import Fraction
# ----- ---------------------------------------------------------
#<------ COMPUTE THE DATA -------------
f1, f2, f3, f4 = 4,2,1,0                                         # <-- choose your representation here
su4_df, st_df   = su4_branching.racah_su4_to_st(f1, f2, f3, f4)  # <--show the dataframe

###-------------------------------export-representation to fomat latex-cvs------------------
### Single representation
#files = export_su4_with_labels(f1, f2, f3, f4, su4_branching)    #<-only one representation 

### Multiple representations
#reps = [(4,2,0,0),(4,2,0,0),(4,1,1,0),(5,3,2,0), (8,4,2,0), (9,4,2,0)]                         
#results = export_multiple_representations(reps, su4_branching)  #<---multiple representations

✓ Module imported successfully!
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,4,2,1,0,],(,5/2,3/2,1/2,),(,2,1,1,),87.0,140



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,1/2,1/2,1,4,4
1,1/2,3/2,2,16,20
2,1/2,5/2,1,12,32
3,3/2,1/2,2,16,48
4,3/2,3/2,2,32,80
5,3/2,5/2,1,24,104
6,5/2,1/2,1,12,116
7,5/2,3/2,1,24,140


*SU(4) to SU(2)_S ⊗ SU(2)_T decomposition from U(6) irrep (ds-shell).*

In [17]:
########################################################################################
##Calculate SU(4) to SU(2)_S ⊗ SU(2)_T decomposition from U(6) irrep                  ##
## for (ds-shell)                                                                     ## 
## Developed by J.P. Valencia, S. Cordoba, and R. Henao                               ##                      
##Convert U(6) Young tableau to SU(4) irrep via conjugate partition.                  ##   
##    Args:                                                                           ##
##        u6_irrep: U(6) irrep as 6 non-increasing non-negative integers              ##       
##    Returns:                                                                        ##
##        (f1, f2, f3, f4) - SU(4) irrep labels                                       ##
##    Raises:                                                                         ## 
##        SymmetryError: If input violates U(6) constraints                           ##
##      Enforces Pauli exclusion principle constraint: f1 <= 4                        ##
##    (first column of conjugate Young diagram cannot exceed 4 boxes)                 ## 
##    Args:                                                                           ##
##        u6_irrep: Tuple of 6 non-negative integers in non-increasing order          ##
##    Returns:                                                                        ##
##     Tuple of 4 integers (f1, f2, f3, f4) representing SU(4) irrep                  ##
##     Raises:                                                                        ##
##     SymmetryError: If f1 > 4 (violates Pauli exclusion principle constraint)       ## 
##     Physical interpretation:                                                       ##
##    - f1 corresponds to the first column height in the conjugate Young diagram      ##
##    - f1 <= 4 is required because SU(4) fundamental representation has dimension 4  ##
##    - This constraint arises from fermionic antisymmetry (Pauli exclusion principle ##
########################################################################################

    
class SymmetryError(ValueError):
    """Exception for invalid symmetry inputs."""
    pass


def u6_to_su4_irrep(u6_irrep: Tuple[int, ...]) -> Tuple[int, int, int, int]:    
    # Validate input format and constraints
    if len(u6_irrep) != 6 or any(not isinstance(x, int) or x < 0 for x in u6_irrep):
        raise SymmetryError("Input must be 6 non-negative integers")
    
    if list(u6_irrep) != sorted(u6_irrep, reverse=True):
        raise SymmetryError("Young diagram must be non-increasing sequence")
        
    # CHECK PAULI EXCLUSION PRINCIPLE CONSTRAINT on U(6) FIRST COLUMN: f1 <= 4
    f1_u6 = u6_irrep[0]
    
    if f1_u6 > 4:
        u6_label = "{" + ", ".join(map(str, u6_irrep)) + "}"
        raise SymmetryError(
            f"❌ PAULI EXCLUSION PRINCIPLE VIOLATED ❌\n"
            f"   U(6) irrep: {u6_label}\n"
            f"   f₁ᵁ⁽⁶⁾ = {f1_u6} > 4 NOT ALLOWED\n"
            f"   First row of U(6) Young diagram cannot exceed 4 boxes\n"
            f"   Physical reason: Cannot have >4 nucleons in same spatial state\n"
            f"   (spin-isospin: 2 spin × 2 isospin = 4 quantum states per spatial orbital)"
        )    
    
    # Compute conjugate partition (transpose Young diagram)
    current = np.array(u6_irrep, dtype=np.int32)
    su4_irrep = []
    
    while np.any(current > 0):
        # Count non-zero elements in current column (height of column)
        non_zero_count = np.count_nonzero(current)
        su4_irrep.append(non_zero_count)
        # Remove first column and decrement remaining elements
        current = current[current > 0] - 1
    
    # Simplify SU(4) irrep by removing columns of length 4 or greater
    while (len(su4_irrep)>=4):
              last_element=su4_irrep[-1]
              su4_irrep = [x - last_element for x in su4_irrep[:-1]]

    f1 = su4_irrep[0] if len(su4_irrep) > 0 else 0    
    # Pad with zeros to ensure exactly 4 elements
    su4_irrep.extend([0] * (4 - len(su4_irrep)))
    
    f1, f2, f3, f4 = tuple(int(x) for x in su4_irrep[:4])
    
    return f1, f2, f3, f4


#################################################################################
##    Calculate SU(4) → SU(2)_S ⊗ SU(2)_T branching from U(6) irrep.           ##
##    This function automates the conversion from orbital U(6) classification  ## 
##    to spin-isospin SU(4) decomposition, then computes all (S,T) multiplets  ##
##    and their multiplicities.                                                ##
##    Args:                                                                    ##
##        U6_irrep: U(6) irrep specified as 6 non-negative integers in         ##
##                 non-increasing order (Young tableau format)                 ## 
##    Returns:                                                                 ##
##        pd.DataFrame with columns:                                           ## 
##            - (S, T): Spin-isospin quantum numbers                           ##
##            - Multiplicity: Inner multiplicity N^{S,T}_{f1,f2,f3,0}          ##
##            - Dimension: (2S+1)(2T+1)                                        ## 
##    Raises:                                                                  ##
##        SymmetryError: If U6_irrep violates Young tableau constraints        ##
##        ValueError: If SU(4) irrep produces no valid branching               ##
##   Examples:                                                                 ##
##        >>> U6_irrep = (4, 2, 0, 0, 0, 0)  # d-shell configuration           ##
##        >>> result_df = U6_RacahSU4toST(U6_irrep)                            ## 
##        >>> print(result_df)                                                 ##        
##    Notes:                                                                   ##
##        - Uses conjugate partition to convert U(6) Young diagram to SU(4)    ##
##        - Enforces SU(4) constraint: first column ≤ 4 boxes (2S+1 ≤ 4)       ##
##        - All results validated via dimensional consistency check            ## 
##      Developed by J.P. Valencia, S. Cordoba, and R. Henao                   ##
#################################################################################

def U6_RacahSU4toST(U6_irrep: Tuple[int, int, int, int, int, int]) -> pd.DataFrame:        
    # Step 1: Convert U(6) irrep to SU(4) labels
    f1, f2, f3, f4 = u6_to_su4_irrep(U6_irrep)
    
    # Step 2: Validate SU(4) irrep (redundant but explicit)
    if not (f1 >= f2 >= f3 >= f4 >= 0):
        raise SymmetryError(f"Invalid SU(4) irrep: ({f1}, {f2}, {f3}, {f4})")
    
    # Step 3: Call branching calculation
    try:
        branching_results = su4_branching.racah_su4_to_st(f1, f2, f3, f4)
    except Exception as e:
        raise ValueError(
            f"Failed to compute SU(4)→(S,T) branching for [{f1},{f2},{f3},{f4}]: {e}"
        )
    
    # Step 4: Format output DataFrame
    if isinstance(branching_results, tuple) and len(branching_results) == 2:
        su4_df, st_df = branching_results
        # Use the ST dataframe (contains spin-isospin decomposition)
        output_df = st_df.copy()
    else:
        output_df = branching_results
    
    # Add metadata columns for traceability
    output_df.attrs['U6_irrep'] = tuple(U6_irrep)
    output_df.attrs['SU4_irrep'] = (f1, f2, f3, f4)
    output_df.attrs['N_total'] = sum(U6_irrep)  
    
    return output_df
# ============================================================================
# EXAMPLE USAGE
# ============================================================================

if __name__ == "__main__":
    import sys
    
    print("=" * 70)
    print("SU(4) Branching Calculator: U(6) → SU(4) → SU(2)_S ⊗ SU(2)_T")
    print("=" * 70)
    
    # Example 1: 
    u6_ex1 = (4, 0, 0, 0, 0, 0) # four nucleons irrep
    print(f"\nExample 1: U(6) irrep {u6_ex1}")
    print("-" * 70)
    try:
        df1 = U6_RacahSU4toST(u6_ex1)

    except Exception as e:
        print(f"Error: {e}")
    
    # Example 2: 
    u6_ex2 = (3, 2, 1, 0, 0, 0) # more complex conjugate irrep
    print(f"\n\nExample 2: U(6) irrep {u6_ex2}")
    print("-" * 70)
    try:
        df2 = U6_RacahSU4toST(u6_ex2)
    except Exception as e:
        print(f"Error: {e}")

    # Example 3: 
    u6_ex3 = (7, 4, 3, 1, 0, 0)   # Pauli exclusion principle violated:  error
    print(f"\n\nExample 3: U(6) irrep {u6_ex3}")
    print("-" * 70)
    try:
        df3 = U6_RacahSU4toST(u6_ex3)
    except Exception as e:
        print(f"Error: {e}")    
    
    print("\n" + "=" * 70)

    # Example 4: 
    u6_ex4 = (4, 4, 3, 1, 0, 0, 0) # len U(6) > 6 :error
    print(f"\n\nExample 4: U(6) irrep {u6_ex4}")
    print("-" * 70)
    try:
        df4 = U6_RacahSU4toST(u6_ex4)
    except Exception as e:
        print(f"Error: {e}")    

    # Example 5: 
    u6_ex5 = (1, 1, 1, 1, 1, 2) # f_6>F_5 :error
    print(f"\n\nExample 5: U(6) irrep {u6_ex5}")
    print("-" * 70)
    try:
        df5 = U6_RacahSU4toST(u6_ex5)
    except Exception as e:
        print(f"Error: {e}") 
    print("\n" + "=" * 70)

SU(4) Branching Calculator: U(6) → SU(4) → SU(2)_S ⊗ SU(2)_T

Example 1: U(6) irrep (4, 0, 0, 0, 0, 0)
----------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,0,0,0,0,],(,0,0,0,),(,0,0,0,),0.0,1



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,0,1,1,1




Example 2: U(6) irrep (3, 2, 1, 0, 0, 0)
----------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,3,2,1,0,],(,2,1,0,),(,1,1,1,),60.0,64



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,1,1,3,3
1,0,2,1,5,8
2,1,0,1,3,11
3,1,1,2,18,29
4,1,2,1,15,44
5,2,0,1,5,49
6,2,1,1,15,64




Example 3: U(6) irrep (7, 4, 3, 1, 0, 0)
----------------------------------------------------------------------
Error: ❌ PAULI EXCLUSION PRINCIPLE VIOLATED ❌
   U(6) irrep: {7, 4, 3, 1, 0, 0}
   f₁ᵁ⁽⁶⁾ = 7 > 4 NOT ALLOWED
   First row of U(6) Young diagram cannot exceed 4 boxes
   Physical reason: Cannot have >4 nucleons in same spatial state
   (spin-isospin: 2 spin × 2 isospin = 4 quantum states per spatial orbital)



Example 4: U(6) irrep (4, 4, 3, 1, 0, 0, 0)
----------------------------------------------------------------------
Error: Input must be 6 non-negative integers


Example 5: U(6) irrep (1, 1, 1, 1, 1, 2)
----------------------------------------------------------------------
Error: Young diagram must be non-increasing sequence



*SU(4) to SU(2)_S ⊗ SU(2)_T decomposition from U(10) irrep (pf-shell)*

In [18]:
###########################################################################################
##Calculate SU(4) to SU(2)_S ⊗ SU(2)_T decomposition from U(10) irrep (pf-shell)         ##
##Developed by J.P. Valencia, S. Cordoba, and R. Henao                                   ##
##For use in pf-shell nuclear structure calculations                                     ##
##    Convert U(10) Young tableau to SU(4) irrep via conjugate partition.                ##
##    Used for pf-shell nuclei (0f₇/₂, 1p₃/₂, 1p₁/₂, 0f₅/₂).                             ##
##    The U(10) group classifies spatial orbital configurations, while SU(4)             ##
##    describes the spin-isospin quantum numbers (S, T).                                 ##
##    Args:                                                                              ##
##        u10_irrep: U(10) irrep as 10 non-increasing non-negative integers              ##
##                   representing Young tableau row lengths                              ##
##    Returns:                                                                           ## 
##        (f1, f2, f3, f4) - SU(4) irrep labels satisfying f1 ≥ f2 ≥ f3 ≥ f4 ≥ 0         ##
##    Raises:                                                                            ## 
##        SymmetryError: If input violates U(10) Young tableau constraints               ##
##    Notes:                                                                             ## 
##        - U(10) has 10 single-particle orbital states: 0f₇/₂ (8-fold), 1p₃/₂ (2-fold)  ##
##        - For pf-shell: typically consider U(8) or U(10) with practical constraints    ##
##        - Conversion uses conjugate partition (transpose of Young diagram)             ##
##        - SU(4) constraint: f1 ≤ 4 (limited by 2S+1 ≤ 4 for fermionic systems)         ## 
##        - U(10) constraint: f1 ≤ 10 (limited by number of single-particle states)      ##
##    Example:                                                                           ##
##        >>> u10_ex = (3, 2, 1, 0, 0, 0, 0, 0, 0, 0)  # 6 nucleons                      ##
##        >>> f1, f2, f3, f4 = u10_to_su4_irrep(u10_ex)                                  ##
##        >>> print(f"SU(4) irrep: [{f1}, {f2}, {f3}, {f4}]")                            ##
###########################################################################################    

def u10_to_su4_irrep(u10_irrep: Tuple[int, ...]) -> Tuple[int, int, int, int]:
    # Validate input format and constraints
    if len(u10_irrep) != 10 or any(not isinstance(x, int) or x < 0 for x in u10_irrep):
        raise SymmetryError("Input must be 10 non-negative integers")
    
    if list(u10_irrep) != sorted(u10_irrep, reverse=True):
        raise SymmetryError("Young diagram must be non-increasing sequence")

    # CHECK PAULI EXCLUSION PRINCIPLE CONSTRAINT on U(10) FIRST COLUMN: f1 <= 4
    f1_u10 = u10_irrep[0]    
    
    if f1_u10 > 4:
        u10_label = "{" + ", ".join(map(str, u10_irrep)) + "}"
        raise SymmetryError(
            f"❌ PAULI EXCLUSION PRINCIPLE VIOLATED ❌\n"
            f"   U(10) irrep: {u10_label}\n"
            f"   f₁ᵁ⁽10⁾ = {f1_u10} > 4 NOT ALLOWED\n"
            f"   First row of U(10) Young diagram cannot exceed 4 boxes\n"
            f"   Physical reason: Cannot have >4 nucleons in same spatial state\n"
            f"   (spin-isospin: 2 spin × 2 isospin = 4 quantum states per spatial orbital)"
        )   

    # Compute conjugate partition (transpose Young diagram)
    current = np.array(u10_irrep, dtype=np.int32)
    su4_irrep = []
    
    while np.any(current > 0):
        # Count non-zero elements in current column (height of column)
        non_zero_count = np.count_nonzero(current)
        su4_irrep.append(non_zero_count)
        # Remove first column and decrement remaining elements
        current = current[current > 0] - 1

    # Simplify SU(4) irrep by removing columns of length 4 or greater
    while (len(su4_irrep)>=4):
              last_element=su4_irrep[-1]
              su4_irrep = [x - last_element for x in su4_irrep[:-1]]
        
    # Remove trailing zeros 
    while len(su4_irrep) > 0 and su4_irrep[-1] == 0:
        su4_irrep.pop()
    
    # Pad with zeros to ensure exactly 4 elements
    su4_irrep.extend([0] * (4 - len(su4_irrep)))
    
    f1, f2, f3, f4 = tuple(int(x) for x in su4_irrep[:4])
    
    return f1, f2, f3, f4

    
    
    

########################################################################################
##    Calculate SU(4) → SU(2)_S ⊗ SU(2)_T branching from U(10) irrep.                 ##
##    This function automates the conversion from orbital U(10) classification        ##
##    (used for pf-shell nuclei) to spin-isospin SU(4) decomposition, then computes   ##
##    all (S,T) multiplets and their multiplicities.                                  ##
## #########################################################
##    Total: 10 single-particle states (hence U(10)                                   ##
##    Args:                                                                           ##
##        U10_irrep: U(10) irrep specified as 10 non-negative integers in             ##
##                  non-increasing order (Young tableau format).                      ##
##                  Example: (5, 3, 2, 0, 0, 0, 0, 0, 0, 0) for 10 nucleons           ##  
##    Returns:                                                                        ## 
##        pd.DataFrame with columns:                                                  ##
##            - (S, T): Spin-isospin quantum numbers                                  ##
##            - Multiplicity: Inner multiplicity N^{S,T}_{f1,f2,f3,0}                 ##
##            - Dimension: (2S+1)(2T+1)                                               ##
##            - Nucleon_Count: Total number of valence nucleons                       ##
##    Raises:                                                                         ##
##        SymmetryError: If U10_irrep violates Young tableau constraints              ## 
##        ValueError: If SU(4) irrep produces no valid branching                      ##
##                                                                                    ##
##    Notes:                                                                          ##
##        - Uses conjugate partition to convert U(10) Young diagram to SU(4)          ##
##        - Enforces SU(4) constraint: first column ≤ 4 boxes (2S+1 ≤ 4)              ##
##        - All results validated via dimensional consistency check                   ##
##        - Typical pf-shell nuclei have 1-10 valence nucleons                        ##
##Developed by J.P. Valencia, S. Cordoba, and R. Henao                                ##
########################################################################################    
def U10_RacahSU4toST(U10_irrep: Tuple[int, int, int, int, int, int, int, int, int, int]) -> pd.DataFrame:    
    # Step 1: Convert U(10) irrep to SU(4) labels
    f1, f2, f3, f4 = u10_to_su4_irrep(U10_irrep)
    
    # Step 2: Validate SU(4) irrep (redundant but explicit)
    if not (f1 >= f2 >= f3 >= f4 >= 0):
        raise SymmetryError(f"Invalid SU(4) irrep: ({f1}, {f2}, {f3}, {f4})")
    
    # Step 3: Call branching calculation
    try:
        branching_results = su4_branching.racah_su4_to_st(f1, f2, f3, f4)
    except Exception as e:
        raise ValueError(
            f"Failed to compute SU(4)→(S,T) branching for [{f1},{f2},{f3},{f4}]: {e}"
        )
    
    # Step 4: Format output DataFrame
    if isinstance(branching_results, tuple) and len(branching_results) == 2:
        su4_df, st_df = branching_results
        # Use the ST dataframe (contains spin-isospin decomposition)
        output_df = st_df.copy()
    else:
        output_df = branching_results
    
    # Step 5: Add metadata for traceability and physics context
   
    output_df.attrs['U10_irrep'] = tuple(U10_irrep)
    output_df.attrs['SU4_irrep'] = (f1, f2, f3, f4)
    output_df.attrs['N_total'] = sum(U10_irrep)
    
  
    return output_df


# ============================================================================
# EXAMPLE USAGE AND BENCHMARKS
# ============================================================================

if __name__ == "__main__":
    
    print("=" * 80)
    print("SU(4) Branching Calculator (pf-shell): U(10) → SU(4) → SU(2)_S ⊗ SU(2)_T")
    print("=" * 80)
    
    # Example 1: 
    u10_ex1 = (4, 2, 0, 0, 0, 0, 0, 0, 0, 0)
    print(f"\nExample 1: U(10) irrep {u10_ex1} (6 nucleons)")
    print("-" * 80)
    try:
        df1 = U10_RacahSU4toST(u10_ex1)
        
    except Exception as e:
        print(f"Error: {e}")
    
    # Example 2:
    u10_ex2 = (4, 1, 1, 0, 0, 0, 0, 0, 0, 0)
    print(f"\n\nExample 2: U(10) irrep {u10_ex2} ")
    print("-" * 80)
    try:
        df2 = U10_RacahSU4toST(u10_ex2)
    
    except Exception as e:
        print(f"Error: {e}")
    
    # Example 3: 
    u10_ex3 = (3, 3, 0, 0, 0, 0, 0, 0, 0, 0)
    print(f"\n\nExample 3: U(10) irrep {u10_ex3}")
    print("-" * 80)
    try:
        df3 = U10_RacahSU4toST(u10_ex3)
        
    except Exception as e:
        print(f"Error: {e}")

    # Example 4: 
    u10_ex4 = (3, 2, 1, 0, 0, 0, 0, 0, 0, 0)
    print(f"\n\nExample 4: U(10) irrep {u10_ex4} ")
    print("-" * 80)
    try:
        df4 = U10_RacahSU4toST(u10_ex4)
        
        if len(df4) > 10:
            print(f"... ({len(df4) - 10} more rows)")
    except Exception as e:
        print(f"Error: {e}")
        
    # Example 5: 
    u10_ex5 = (3, 1, 1, 1, 0, 0, 0, 0, 0, 0)
    print(f"\n\nExample 5: U(10) irrep {u10_ex5} ")
    print("-" * 80)
    try:
        df5 = U10_RacahSU4toST(u10_ex5)
        
        if len(df5) > 10:
            print(f"... ({len(df5) - 10} more rows)")
    except Exception as e:
        print(f"Error: {e}")
        
# Example 6: 
    u10_ex6 =(2, 2, 2, 0, 0, 0, 0, 0, 0, 0)# 
    print(f"\n\nExample 6: U(10) irrep {u10_ex6} ")
    print("-" * 80)
    try:
        df6 = U10_RacahSU4toST(u10_ex6)
        
        if len(df6) > 10:
            print(f"... ({len(df6) - 10} more rows)")
    except Exception as e:
        print(f"Error: {e}")
    
# Example 7: 
    u10_ex7 =(2, 2, 1, 1, 0, 0, 0, 0,0, 0)
    print(f"\n\nExample 7: U(10) irrep {u10_ex7} ")
    print("-" * 80)
    try:
        df7 = U10_RacahSU4toST(u10_ex7)
        
        if len(df7) > 10:
            print(f"... ({len(df7) - 10} more rows)")
    except Exception as e:
        print(f"Error: {e}")

# Example 8: 
    u10_ex8 = (1, 1, 1, 1, 1, 1, 0, 0, 0, 0)
    print(f"\n\nExample 8: U(10) irrep {u10_ex8} ")
    print("-" * 80)
    try:
        df8 = U10_RacahSU4toST(u10_ex8)
        
        if len(df8) > 10:
            print(f"... ({len(df8) - 10} more rows)")
    except Exception as e:
        print(f"Error: {e}")        

SU(4) Branching Calculator (pf-shell): U(10) → SU(4) → SU(2)_S ⊗ SU(2)_T

Example 1: U(10) irrep (4, 2, 0, 0, 0, 0, 0, 0, 0, 0) (6 nucleons)
--------------------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,1,1,0,0,],(,1,0,0,),(,0,1,0,),20.0,6



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,1,1,3,3
1,1,0,1,3,6




Example 2: U(10) irrep (4, 1, 1, 0, 0, 0, 0, 0, 0, 0) 
--------------------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


...

,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,0,1,1,1
1,0,2,1,5,6
2,1,1,2,18,24
3,1,2,1,15,39
4,1,3,1,21,60
5,2,0,1,5,65
6,2,1,1,15,80
7,2,2,1,25,105
8,3,1,1,21,126




Example 8: U(10) irrep (1, 1, 1, 1, 1, 1, 0, 0, 0, 0) 
--------------------------------------------------------------------------------
● SU(4) Representation Info:(irrep notations, casimir order two, irrep dimension)


,[,f1,f2,f3,f4,],(,p1,p2,p3,),(,α,β,γ,),C_2[SU(4)],dimension
0,[,6,0,0,0,],(,3,3,3,),(,6,0,0,),180.0,84



● Branching Rules to (S, T):


,Spin,Isospin,mult,"dim (S,T)","cum_sum_dim(S,T)"
0,0,0,1,1,1
1,1,1,1,9,10
2,2,2,1,25,35
3,3,3,1,49,84
